# 第11章：ロジスティック回帰

## 11.2 ロジスティック回帰の実装

### 11.2.1 モジュールなどの宣言

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from sklearn.preprocessing import StandardScaler # 標準化用モジュール
from sklearn.model_selection import train_test_split # ホールドアウト用モジュール
from sklearn.linear_model import LogisticRegression as logr # ロジスティック回帰用モジュール
from sklearn.metrics import accuracy_score, roc_auc_score # 評価指標用モジュール

### 11.2.2 pickleファイルの読み込みと訓練データの分割

In [ ]:
# 前処理済み中間データのdictを読み取る
pp_data_dict = joblib.load("../intermediate/pp_data_dict.pkl3")

# dictのvalueに格納されたロジスティック回帰向け中間データを読み取る
lgr_train_df = pp_data_dict["lgr"]["train"]
lgr_test_df = pp_data_dict["lgr"]["test"]

# 6:4の割合でホールドアウト法を行う
lgr_train_train_df, lgr_train_valid_df = train_test_split(lgr_train_df, test_size=0.4, random_state=57, shuffle=True)

lgr_train_df.shape, lgr_train_train_df.shape, lgr_train_valid_df.shape, lgr_test_df.shape

In [ ]:
# 目的変数をtargetという変数に格納する
target = "buy_flag"

# 説明変数をfeaturesという変数に格納する
lgr_features = lgr_train_df.columns.tolist()

# customer_idとbuy_flagは説明変数ではない為削除する
lgr_features.remove("customer_id")
lgr_features.remove("buy_flag")

## 11.3 ハイパーパラメータチューニング

### 11.3.2 グリッドサーチ

In [ ]:
# ハイパーパラメータチューニング
# グリッドサーチ
# 探索するハイパーパラメータの候補をリストに格納する.
lgr_params_list = [{"penalty": "l2", "C": ii, "random_state": 57} for ii in np.arange(0.005, 0.5, 0.005)]
lgr_params_list

In [ ]:
# グリッドサーチの結果を格納するリスト
gs_result_list = []

# グリッドサーチの実施
for params_dict in lgr_params_list:
    # 定義する
    tmp_lgr_clf = logr(**params_dict, max_iter=1000)
    # 学習する
    tmp_lgr_clf.fit(lgr_train_train_df[lgr_features], lgr_train_train_df[target])
    
    # train-trainを予測する（predict関数）
    train_pred_y = tmp_lgr_clf.predict(lgr_train_train_df[lgr_features])
    
    # train-trainを予測する（predict_proba関数）
    train_proba_y = tmp_lgr_clf.predict_proba(lgr_train_train_df[lgr_features]).T[1]
    
    # train-validを予測する（predict関数）
    valid_pred_y = tmp_lgr_clf.predict(lgr_train_valid_df[lgr_features])
    
    # train-validを予測する（predict_proba関数）
    valid_proba_y = tmp_lgr_clf.predict_proba(lgr_train_valid_df[lgr_features]).T[1]
    
    # train-trainを検証する
    train_accuracy_val = accuracy_score(lgr_train_train_df[target], train_pred_y)
    train_auc_val = roc_auc_score(lgr_train_train_df[target], train_proba_y)
    
    # train-validを検証する
    valid_accuracy_val = accuracy_score(lgr_train_valid_df[target], valid_pred_y)
    valid_auc_val = roc_auc_score(lgr_train_valid_df[target], valid_proba_y)
    
    # リストに格納する
    gs_result_list += [[params_dict, params_dict["C"], train_accuracy_val, train_auc_val, valid_accuracy_val, valid_auc_val]]

In [ ]:
# 結果をDataFrameに格納する
gs_result_df = pd.DataFrame(gs_result_list, columns=["params_dict", "L2_regularization_strength", "train_accuracy", "train_auc", "valid_accuracy", "valid_auc"])
# 各パラメータ値ごとのvalidのAUCを確認する
gs_result_df[["L2_regularization_strength", "train_accuracy", "train_auc", "valid_accuracy", "valid_auc"]].sort_values(by="valid_auc", ascending=False).reset_index(drop=True).head(10)

### 11.3.3 ロジスティック回帰の可視化

In [ ]:
# Accuracyの可視化
plt.figure(figsize=(6, 5), facecolor="white", dpi=150)
# train Accuracy
plt.plot(gs_result_df["L2_regularization_strength"], gs_result_df["train_accuracy"], label="train_accuracy")
# valid Accuracy
plt.plot(gs_result_df["L2_regularization_strength"], gs_result_df["valid_accuracy"], label="valid_accuracy")
plt.title("Accuracy\nlgr L2_regularization_strength tuning", fontsize=10)
plt.xlabel("L2_regularization_strength", fontsize=10)
plt.ylabel("Accuracy", fontsize=10)
plt.legend()
plt.grid()
plt.show()

In [ ]:
# AUCの可視化
plt.figure(figsize=(6, 5), facecolor="white", dpi=150)
# train AUC
plt.plot(gs_result_df["L2_regularization_strength"], gs_result_df["train_auc"], label="train_auc")
# valid AUC
plt.plot(gs_result_df["L2_regularization_strength"], gs_result_df["valid_auc"], label="valid_auc")
plt.title("AUC\nlgr L2_regularization_strength tuning", fontsize=10)
plt.xlabel("L2_regularization_strength", fontsize=10)
plt.ylabel("AUC", fontsize=10)
plt.legend()
plt.grid()
plt.show()

In [ ]:
# train-validのAUCが最も高いハイパーパラメータの組み合わせを変数に格納する.
lgr_best_idx = np.argmax(gs_result_df["valid_auc"])
lgr_best_params = gs_result_df["params_dict"].values[lgr_best_idx]
lgr_best_score = gs_result_df["valid_auc"].values[lgr_best_idx]

# 内容を確認
print("lgr_best_params:", lgr_best_params)
print("lgr_best_score:", lgr_best_score)

## 11.4 最適化したロジスティック回帰モデルの実装

### 11.4.1 モデルの構築

In [ ]:
# 最適化したハイパーパラメータを使用してモデルを定義する
lgr_clf = logr(**lgr_best_params, max_iter=1000)

# 学習する
lgr_clf.fit(lgr_train_df[lgr_features], lgr_train_df[target])

### 11.4.2 標準化の実施

In [ ]:
# 標準化されていないことを確認する
lgr_train_df[lgr_features].describe()

In [ ]:
# 説明変数の標準化を行う
scaler = StandardScaler()
lgr_train_df[lgr_features] = scaler.fit_transform(lgr_train_df[lgr_features])

# 標準化されたことを確認する
lgr_train_df[lgr_features].describe()

In [ ]:
# 最適化したハイパーパラメータを使用してモデルを定義する
lgr_clf = logr(**lgr_best_params, max_iter=1000)

# 学習する
lgr_clf.fit(lgr_train_df[lgr_features], lgr_train_df[target])

### 11.4.3 偏回帰係数の確認

In [ ]:
# 偏回帰係数を確認する
lgr_coef_df = pd.DataFrame({"feature": lgr_features, "coefficient": lgr_clf.coef_[0]})
lgr_coef_df = lgr_coef_df.append(pd.DataFrame({"feature": "constant", "coefficient": lgr_clf.intercept_}))
lgr_coef_df["abs_coefficient"] = lgr_coef_df["coefficient"].abs()
lgr_coef_df = lgr_coef_df.sort_values(by="abs_coefficient", ascending=False).reset_index(drop=True)
lgr_coef_df.head(15)

### 11.4.4 予測と検証

In [ ]:
# trainを予測する
train_pred_y = lgr_clf.predict(lgr_train_df[lgr_features])
train_proba_y = lgr_clf.predict_proba(lgr_train_df[lgr_features]).T[1]

# trainを検証する
train_accuracy_val = accuracy_score(lgr_train_df[target], train_pred_y)
train_auc_val = roc_auc_score(lgr_train_df[target], train_proba_y)
train_accuracy_val, train_auc_val

### 11.4.5 テストデータの予測

In [ ]:
# testを予測する
test_pred_y = lgr_clf.predict(lgr_test_df[lgr_features])
test_proba_y = lgr_clf.predict_proba(lgr_test_df[lgr_features]).T[1]

In [ ]:
# sample submitデータを読み込む
gi_sample_submit_df = pd.read_csv("../input/gi_sample_submit.csv")

In [ ]:
# submit向けDataFrameを作成し、列に予測確率を格納する
submit_df = lgr_test_df.copy()[["customer_id"]]
submit_df["buy_proba"] = test_proba_y
submit_df.head()

In [ ]:
gi_sample_submit_df.shape, submit_df.shape

In [ ]:
submit_df = pd.merge(gi_sample_submit_df.drop("buy_proba", axis=1), submit_df, on="customer_id", how="left").reset_index(drop=True)
submit_df.head()

In [ ]:
submit_df.shape

In [ ]:
# outputディレクトリにsubmit用ファイルを出力する
submit_df.to_csv(f"../output/submit_lgr.csv", encoding="utf-8", index=False)